In [1]:
# coding: utf-8
# ----------------------------------------------------------------------------
# Name:        pdf_functions.py
# Purpose:     Functions to create 2019 grower report PDFs.
#
# Author:      German Mandrini
# Created:     2019
# ----------------------------------------------------------------------------

import os
import time
import json
import tempfile
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import matplotlib
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_RIGHT
from reportlab.lib.pagesizes import letter, landscape
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import Paragraph, Table, Image, Frame, Spacer
from io import BytesIO
from os import path, walk


ModuleNotFoundError: No module named 'reportlab'

In [2]:
def cap(string):
    return " ".join([w[0].upper() + w[1:] for w in string.split()])


def copyright_footer(c, year=2019, offset=5.5, font="Helvetica", split=False):
    """Add copyright footer to a reportlab canvas."""
    # define a font
    c.setFont(font, 12)
    if split:
        copyright_str = "© {} The Climate Corporation.".format(year)
        copyright_str_2 = "All Rights Reserved."
        c.drawCentredString(inch * offset, inch * 0.45, copyright_str)
        c.drawCentredString(inch * offset, inch * 0.25, copyright_str_2)
    else:
        copyright_str = "© {} The Climate Corporation. All Rights Reserved.".format(year)
        c.drawCentredString(inch * offset, inch * 0.3, copyright_str)

styles = getSampleStyleSheet()
font="Helvetica"
font_bold="Helvetica-Bold" 
# Add a custom heading font
styles.add(
ParagraphStyle(
    'Heading_Custom_18pt_center',
    parent=styles['Normal'],
    fontName=font_bold,
    fontSize=18,
    leading=21.6,
    alignment=TA_CENTER
)
)

# Add a heading font with less spacing
styles.add(
ParagraphStyle(
    'Heading_Custom_18pt_center_slim_italic',
    parent=styles['Normal'],
    fontName="Helvetica-BoldOblique",
    fontSize=18,
    leading=21.6,
    alignment=TA_LEFT
)
)

# Add a custom paragraph font
styles.add(
ParagraphStyle(
    'Normal_Custom_14pt_center',
    parent=styles['Normal'],
    fontSize=14,
    leading=20,
    alignment=TA_LEFT
)
)

# Make variables to hold custom fonts
style_normal_custom_14pt = styles['Normal_Custom_14pt_center']
style_heading_custom = styles['Heading_Custom_18pt_center']
style_heading_slim = styles['Heading_Custom_18pt_center_slim_italic']

styles = getSampleStyleSheet()

# Add a custom paragraph font
styles.add(
    ParagraphStyle(
        'Normal_Custom_11pt',
        parent=styles['Normal'],
        fontSize=11,
        leading=13.2
    )
)

# Add a custom paragraph font
styles.add(
    ParagraphStyle(
        'Normal_Custom_8pt',
        parent=styles['Normal'],
        fontSize=8,
        leading=9.6
    )
)

# Add a custom title font
styles.add(
    ParagraphStyle(
        'Normal_Custom_16pt_center',
        parent=styles['Normal'],
        fontSize=16,
        leading=19.2,
        alignment=TA_CENTER,
        spaceBefore=0,
        spaceAfter=0
    )
)

styles.add(
    ParagraphStyle(
        'Normal_Custom_12pt',
        parent=styles['Normal'],
        fontSize=12,
        leading=14.4
    )
)

# Make variables to hold custom fonts
style_normal = styles['Normal']
style_normal_custom_11pt = styles['Normal_Custom_11pt']
style_normal_custom_16pt_center = styles['Normal_Custom_16pt_center']
style_normal_custom_11pt = styles['Normal_Custom_11pt']
style_normal_custom_8pt = styles['Normal_Custom_8pt']
style_normal_custom_12pt = styles['Normal_Custom_12pt']

In [3]:
def make_grower_resport(field_n):
    # FIRST FOLLOW generate_hpt_grower_report_pdf
    # Make canvas, and add to it page by page, and save PDF. Intended for variable-rate protocols.
    pdf_data = BytesIO()
    save_name = 'data/growers_reports/reports_pdf/field_'+ field_n +'.pdf'
    c = canvas.Canvas(save_name, pagesize=landscape(letter))
    # ~~~~~~~~~~~~~~~~~~~~~~
    # Page 1
    # Make canvas
    # THIS COMES FROM page1_canvas_hpt

    # =============================================================== Page 1 HPT
    # def page1_canvas_hpt(c,
    pair_number=1

    year=2019
    bucket="com.climate.production.analytics"


    # Set Line color
    c.setStrokeColorRGB(.2, .2, .2)  # dark gray
    #=================================================================================================#
    #LOAD THE GENERAL INFO OF THE FIELD
    general_info_name = "data/growers_reports/data_table/"+field_n+"_table.json"

    with open(general_info_name) as json_file:  
        general_info = json.load(json_file)

    #=================================================================================================#
    # Add header and title

    logos_dict = {'OH': 'sunrise_logo.png',
                    'IL': 'growmark_logo.png',
                    'IA': 'growmark_logo.png'}
    
    #---- Retailer logo

    state_tmp = general_info['state']
    if state_tmp in logos_dict:
        logo_retailer = "data/growers_reports/logos/"+logos_dict[state_tmp]
    else:
        logo_retailer = "data/growers_reports/logos/climate_left_logo.png"

    c.drawImage(logo_retailer, inch * 0.1, inch * 7.2, width=inch * 1.8, height=inch * 0.7,
                preserveAspectRatio=True, anchor='c', showBoundary=0)

    #c.rect(inch * 0.1, inch * 7.7, width=inch * 1.8, height=inch * 0.7, fill=0)

    #---- Climate logo
    header = "data/growers_reports/logos/header_fieldview2.png"
    c.drawImage(header, inch * 1.9 , inch * 7.2 , width=inch * 8.8, height=inch * 0.8,
                preserveAspectRatio=True, anchor='c', showBoundary=0)

    #c.rect(inch * 1.9 , inch * 7.7 , width=inch * 9, height=inch * 0.8, fill=0)


    #=================================================================================================#
    #ADD A MAIN TITLE

    f = Frame(inch * 0.3, inch * 6.6, width=inch * 5, height=inch * 0.7, showBoundary=0, topPadding=12)
    f.addFromList([Paragraph(cap('VARIABLE RATE NITROGEN REPORT'), style_heading_custom)], c) # style_heading_slim

    #=================================================================================================#
    # Set up paragraph for left side of the page
    paragraph_spacing_pt = 12
    paragraph = []

    paragraph.append(Paragraph(cap('User : ' + general_info['first_name'] + " " + 
                                   general_info['last_name']), style_heading_slim))

    paragraph.append(Paragraph(cap('Field : ' + general_info['field_name']), style_heading_slim))
    paragraph.append(Paragraph("<b>E-mail</b> - {}".format(general_info['email']), style_normal_custom_12pt))
    # paragraph.append(Paragraph("<b>Field</b> : {}".format(general_info['field_name']), 
    #                            style_normal_custom_12pt))


    paragraph.append(Spacer(1, paragraph_spacing_pt)) # # Add space

    # # Dates
    plant_date = general_info['planting_date2']

    paragraph.append(Paragraph("<b>Plant Date</b> - N/A", style_normal_custom_12pt))

    harvest_date = general_info['harvest_date']

    paragraph.append(Paragraph("<b>Harvest Date</b> - {}".format(harvest_date), style_normal_custom_12pt))


    paragraph.append(Paragraph("<b>Applied Product</b> - {}".format(general_info['product_n']), style_normal_custom_12pt))

    # Create the frame and build the paragraph
    f = Frame(inch * 0.3, inch * 4.6, width=inch * 5, height=inch * 1.8, showBoundary=0)
    f.addFromList(paragraph, c)

    #=================================================================================================#
    # ADD CUMULATIVE PRECIPITATION WITH TITLE

    f = Frame(inch * 0.3, inch * 3.8, width=inch * 5, height=inch * 0.5, showBoundary=0)
    f.addFromList([Paragraph("Cumulative Precipitation", style_heading_custom)], c)


    path_graph_p1 = "data/growers_reports/figures/"+field_n+"_precipitation.png"

    c.drawImage(path_graph_p1, inch * 0.3, inch * 0.5, width=inch * 5.0, height=inch * 3.2,
                preserveAspectRatio=True, anchor='c', showBoundary=0)

    # c.rect(inch * 0.3, inch * 0.5, width=inch * 5.0, height=inch * 3.2, fill=0)

    #=================================================================================================#
    # ADD AS-APPLIED ON THE RIGHT

    f = Frame(inch * 5.7, inch * 6.8, width=inch * 5, height=inch * 0.5, showBoundary=0)
    f.addFromList([Paragraph("As-Applied Map", style_heading_custom)], c)

    path_graph_1 = "data/growers_reports/figures/"+field_n+"_asapp_map.png"
    c.drawImage(path_graph_1, inch * 5.7, inch * 3.9, width=inch * 5, height=inch * 2.9,
                preserveAspectRatio=True, anchor='c', showBoundary = 0)
    # c.rect(inch * 5.7, inch * 4.1, width=inch * 5.0, height=inch * 2.9, fill=0)

    # ----
    # ADD YIELD MAP ON THE RIGHT

    f = Frame(inch * 5.7, inch * 3.4, width=inch * 5, height=inch * 0.5, showBoundary=0)
    f.addFromList([Paragraph("Yield Map", style_heading_custom)], c)

    path_graph_2 = "data/growers_reports/figures/"+field_n+"_yield_map.png"
    c.drawImage(path_graph_2, inch * 5.7, inch * 0.5, width=inch * 5.0, height=inch * 2.9,
                preserveAspectRatio=True, anchor='c', showBoundary = 0)
    # c.rect(inch * 5.7, inch * 0.5, width=inch * 5.0, height=inch * 2.8, fill=0)
    #=================================================================================================#
    # # Add copyright
    copyright_footer(c, font=font, year=year)  # offset=2.35,

    c.showPage()  # This starts new page


    # # ======================================================================================================================= main function
    # def page4_canvas_general(c,
    page_number=4
    year=2019

    #=================================================================================================#
    # Add left bar plot

    paragraph = []
    paragraph.append(Paragraph("Nitrogen and Yield", style_heading_custom))
    f = Frame(inch * 0.3, inch * 7.1, width=inch * 5, height=inch * 0.5, showBoundary=0)
    f.addFromList(paragraph, c)

    path_graph_3 = "data/growers_reports/figures/"+field_n+"_yield_n_bar_v.png"
    c.drawImage(path_graph_3, inch * 0.3, inch * 4, width=inch * 5, height=inch * 3, 
                preserveAspectRatio = True, anchor='n')

    c.rect(inch * 0.3, inch * 4, width=inch * 5, height=inch * 3, fill=0)

    # Add right bar plot
    paragraph = []
    paragraph.append(Paragraph("Partial Profits", style_heading_custom)) #style_normal_custom_16pt_center
    f = Frame(inch * 5.7, inch * 7.1, width=inch * 5, height=inch * 0.5, showBoundary=0)
    f.addFromList(paragraph, c)

    path_graph_4 = "data/growers_reports/figures/"+field_n+"_profits_bar_v.png"
    c.drawImage(path_graph_4, inch * 5.5, inch * 4, width=inch * 5, height=inch * 3, preserveAspectRatio = True, anchor='n')
    c.rect(inch * 5.7, inch * 4, width=inch * 5, height=inch * 3, fill=0)

    #=================================================================================================#
    # Prices explanation
    Pc = general_info["Pc"]
    Pn = general_info["Pn"]

    disclaimer = "Note: Profits were calculated using the formula Profits = Pc * Yld - Pn * N, "\
                 "where Pc is the Price of Corn equal to {} ($/bu) and Pn is the Price of Nitrogen "\
                 "equal to {} ($/lb of Nitrogen)".format(round(Pc,2), round(Pn,2))
    paragraph = []
    paragraph.append(Paragraph(disclaimer, style_normal_custom_11pt))
    f = Frame(inch * 0.3, inch * 3.3, width=inch * 10.4, height=inch * 0.6, showBoundary=0)
    f.addFromList(paragraph, c)


    #=================================================================================================#
    #MAKE A TABLE WITH THE SUMMARY
    general_info_table = general_info['summary']
    json_acceptable_string = general_info_table.replace("'", "\"")
    d = json.loads(json_acceptable_string)
    general_info_table = pd.DataFrame.from_dict(d)    
    general_info_table
    treatment = list(general_info_table.index.values)
    general_info_table['treatment'] = ['Static' if i == 'static_rate_zone' else 'VRN' for i in treatment]

    cols = general_info_table.columns.tolist()

    cols = cols[-1:] + cols[:-1]
    general_info_table = general_info_table[cols]

    general_info_table = general_info_table.round({'area_ac': 1, 'yield_bu_ac': 1, 'n_asapplied': 1, 'p_dol_ac': 2})


    lista = [['Treatment','Area (ac)', 'Yld (bu/ac)', 'N_app (lb/ac)', 'Profits ($/ac)']] + general_info_table.values.tolist()


    ts = [('ALIGN', (1,1), (-1,-1), 'CENTER'),
         ('LINEABOVE', (0,0), (-1,0), 1, colors.black),
         ('LINEBELOW', (0,0), (-1,0), 1, colors.black),
         ('FONT', (0,0), (-1,0), 'Times-Bold'),
         ('LINEABOVE', (0,-1), (-1,-1), 1, colors.black),
          ('LINEBELOW', (0,-1), (-1,-1), 1, colors.black),
          ('FONTSIZE', (0, 0), (-1, -1), 12),
         # ('LINEBELOW', (0,-1), (-1,-1), 0.5, colors.black, 1, None, None, 4,1),
         # ('LINEBELOW', (0,-1), (-1,-1), 1, colors.red),
         # ('FONT', (0,-1), (-1,-1), 'Times-Bold'),
         # ('BACKGROUND',(1,1),(-2,-2),colors.green),
         # ('TEXTCOLOR',(0,0),(1,-1),colors.red)
         ]

    table = Table(lista, style=ts)
    elements = []
    elements.append(table)

    # Create the frame and build the paragraph
    f = Frame(inch * ((11-5)/2), inch * 1.5, width=inch * 5, height=inch * 1.5, showBoundary=0)
    f.addFromList(elements, c)


    #=================================================================================================#

    disclaimer = "Our services provide estimates or recommendations based on models. These do not guarantee results. Consult your agronomist, " \
                 "commodities broker and other service professionals before making financial, risk management and farming decisions. Information " \
                 "and recommendations we provide do not modify your rights under insurance policies purchased through our affiliates. More information " \
                 "at http://www.climate.com/disclaimers. iPad® is a registered mark of Apple, Inc. Climate FieldView™  is a trademark of The Climate " \
                 "Corporation. © {} The Climate Corporation. All Rights Reserved.".format(year)
    paragraph = []
    paragraph.append(Paragraph(disclaimer, style_normal_custom_8pt))
    f = Frame(inch * 0.2, inch * 0.02, width=inch * 10.6, height=inch * 0.7, showBoundary=0)
    f.addFromList(paragraph, c)


    c.showPage()

    c.save()

In [4]:
possible_fields = [a_file.replace('_table.json','')
                   for root, _, files in walk("data/growers_reports/data_table/")
                    for a_file in files
                    if a_file.endswith('.json')]

for count, field_n in enumerate(possible_fields):
    #print(field_n)
    try:
        make_grower_resport(field_n)
    except Exception as e:
        print(e)